# ENTREGABLE 1

INSTRUCCIONES

Realizar la primera fase del análisis exploratorio (limpieza) usando el archivo CSV (dataset_banco.csv) con 45215 filas y 17 columnas.

# 1. El problema del negocio

Una entidad bancaria contrata a una empresa de marketing encargada de contactar telefónicamente a posibles clientes para determinar si están interesados o no en adquirir un certificado de depósito a término con el banco.

¿Qué perfil tienen los clientes con mayor potencial de conversión?

#2. El set de datos

Cada registro contiene 16 características (las primeras 16 columnas) y una categoría ("yes" o "no" dependiendo de si la persona está o no interesada en adquirir el producto). Las columnas son:

1. "age":  edad (numérica)
2. "job": tipo de trabajo (categórica: "admin.", "unknown", "unemployed", "management", "housemaid", "entrepreneur", "student", "blue-collar","self-employed", "retired", "technician", "services")
3. "marital": estado civil (categórica: "married", "divorced", "single")
4. "education": nivel educativo (categórica: "unknown", "secondary", "primary", "tertiary")
5. "default": si dejó de pagar sus obligaciones (categórica: "yes", "no")
6. "balance": saldo promedio anual en euros (numérica)
7. "housing": ¿tiene o no crédito hipotecario? (categórica: "yes", "no")
8. "loan": ¿tiene créditos de consumo? (categórica: "yes", "no")
9. "contact": medio a través del cual fue contactado (categórica: "unknown", "telephone", "cellular")
10. "day": último día del mes en el que fue contactada (numérica)
11. "month": último mes en el que fue contactada (categórica: "jan", "feb", "mar", ..., "nov", "dec")
12. "duration": duración (en segundos) del último contacto (numérica)
13. "campaign": número total de veces que fue contactada durante la campaña (numérica)
14. "pdays": número de días transcurridos después de haber sido contactado antes de la campaña actual (numérica. -1 indica que no fue contactado previamente)
15. "previous": número de veces que ha sido contactada antes de esta campaña (numérica)
16. "poutcome": resultado de la campaña de marketing anterior (categórica: "unknown", "other", "failure", "success")
17. "y": categoría ¿el cliente se suscribió a un depósito a término? (categórica: "yes", "no")

#3. Una primera mirada al dataset

In [ ]:
# Importar librerías
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
ruta = "./dataset_banco.csv"
data = pd.read_csv(ruta)

In [ ]:
print(data.shape)
data.head()

In [ ]:
# Veamos las variables categóricas y las numéricas
data.info()

#4. Limpieza

Realizaremos el proceso de limpieza teniendo en cuenta las situaciones más comunes:

1. Datos faltantes en algunas celdas
2. Columnas irrelevantes (que no responden al problema que queremos resolver)
3. Registros (filas) repetidos
4. Valores extremos (*outliers*) en el caso de las variables numéricas. Se deben analizar en detalle pues no necesariamente la solución es eliminarlos
5. Errores tipográficos en el caso de las variables categóricas

Al final de este proceso de limpieza deberíamos tener un set de datos **íntegro**, listo para la fase de Análisis Exploratorio.

## 4.1 Datos faltantes

Comenzamos a ver que los datos no están completos, pues no todas las columnas tienen la misma cantidad de registros.

El número total de registros debería ser 45.215. Sin embargo columnas como "job", "marital", "education", "balance", "duration" y "pdays".

Por ser tan pocos los datos  faltantes optaremos por eliminar las filas correspondientes:

In [ ]:
valores_nulos = data.isnull().sum()

print(f'Null values per colum \n{valores_nulos}')

 As you can see there are too few null values, so we proced to delete them

In [ ]:
data_1 = data.dropna()

print(data_1.isnull().sum())

##4.2 Columnas irrelevantes

Una columna irrelevante puede ser:

- **Una columna que no contiene información relevante para el problema que queremos resolver**. Por ejemplo en este caso podría ser una columna que no guarde relación con el posible perfil del cliente (deporte favorito, hobbies, comida favorita, etc.)
- **Una columna categórica pero con un sólo nivel**. Por ejemplo si en la columna "job" solo tuviésemos el nivel "unknown".
- **Una columna numérica pero con un sólo valor**. Por ejemplo si en la columna "edad" todos los vlaores fuesen iguales a 50.
- **Columnas con información redundante**. Por ejemplo si además de las columnas "month" y "day" tuviésemos la columna "month-day", resultado de combinar las dos anteriores.

Si tenemos la duda de si alguna columna puede ser relevante o no lo mejor es dejarla (y más adelante en posteriores etapas podremos darnos cuenta de si se debe preservar o no).

En este caso todas las columnas pueden resultar relevantes, pero debemos verificar que no haya columnas categóricas con un sólo nivel, o columnas numéricas con un sólo valor:

Todas las columnas categóricas tienen más de 1 subnivel. No eliminaremos ninguna.

Verifiquemos lo que ocurre con las columnas numéricas:

Todas las columnas numéricas tienen desviaciones estándar ("std") diferentes de cero, lo que indica que no tienen un único valor.

Preservaremos todas las columnas numéricas.

we chech if there are any column with one only value.

In [ ]:
categorical_cols = data_1.select_dtypes(include=['object', 'category']).columns
cols_to_drop = [col for col in categorical_cols if data_1[col].nunique() == 1]

numeric_cols = data_1.select_dtypes(include=['number']).columns
cols_to_drop += [col for col in numeric_cols if data_1[col].nunique() == 1]

cols_to_drop += [col for col in numeric_cols if data_1[col].std() == 0]

print(f'columns to drop: {cols_to_drop}')

##4.3 Filas repetidas

In [ ]:
duplicate_rows = data_1.duplicated().sum()
data_2 = data_1.drop_duplicates()

print("Número de filas duplicadas:", duplicate_rows)

##4.4 *Outliers* en las variables numéricas

No siempre se deben eliminar los *outliers* porque dependiendo de la variable numérica analizada estos pueden contener información importante.


In [ ]:
no_sense_age = data_2[(data_2['age'] > 100) | (data_2['age'] < 18)]
no_sense_age_i = data_2[(data_2['age'] > 100) | (data_2['age'] < 18)].index
data_3 = data_2.drop(no_sense_age_i)

print(f"number rows with no sense age: {len(no_sense_age)}")

In [ ]:
no_sense_duration = data_3[(data_2['duration'] < 0)]
no_sense_duration_i = data_3[(data_2['duration'] < 0)].index
data_4 = data_3.drop(no_sense_duration_i)

print(f"number rows with no sense age: {len(no_sense_duration)}")

##4.5 Errores tipográficos en variables categóricas

En una variable categórica pueden aparecer sub-niveles como "unknown" y "UNK" que para nosotros son equivalentes pero que para nuestro programa parecerían diferentes.

Se deben unificar estos sub-niveles

In [ ]:
reparto_job = data_4['job'].value_counts()

print(reparto_job)

As we can see, there are lots of nonsens values, as having admin. AND administrative or Management and MANAGEMENT

In [ ]:
data_5 = data_4.copy()
data_5['job'] = data_5['job'].replace({
    'administrative': 'admin.',
    'Management': 'management',
    'MANAGEMENT': 'management',
    'Services': 'services',
    'Self-employed': 'self-employed',
    'Retired': 'retired'
})
print(data_5['job'].value_counts())

As you can se we have the column unkonwn, this must be converted as a null value.

The othe columns have almost the same problem as this one, same value called with other name.

I am not going to clean them all because it will be repetitive and this is a how-to-do notebook.

## Conclusiones

This is the mandatory first step, skip this step will make the rest of the work useless, and as you have seen when you are cleaning you will be noticing of new things to do, as we have seen with the unknown value in the job column

In [ ]:
#data.to_csv(ruta, index=False)